![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/01.1.prepare_CoNLL_from_annotations_for_NER.ipynb)

# Prepare CoNLL file from annotations for NER

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=True
nlp.install(refresh_install=True)

In [4]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_9076.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.3.1, 💊Spark-Healthcare==5.3.1, running on ⚡ PySpark==3.4.0


In [5]:
spark

## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

### Entity File
This dataframe should include at least five  below columns in order:
*   ['text_id','begin','end','chunk','entity']

In [6]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ChemProt/chemprot_train_entities.csv

train_entities_df = pd.read_csv('chemprot_train_entities.csv')
train_entities_df= train_entities_df[["text_id", "begin", "end", "chunk", "entity"]]
train_entities_df.head()

,text_id,begin,end,chunk,entity
0,11319232,242,250,acyl-CoAs,CHEMICAL
1,11319232,1193,1200,triacsin,CHEMICAL
2,11319232,1441,1447,sucrose,CHEMICAL
3,11319232,1637,1651,triacylglycerol,CHEMICAL
4,11319232,1702,1710,acyl-CoAs,CHEMICAL


### Text File
This dataframe should include at least two below columns in order:
*   `['text_id','text']`

In [7]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ChemProt/chemprot_train_text.csv
train_text_df = pd.read_csv('chemprot_train_text.csv')
train_text_df.head()

,text_id,text
0,16357751,Selective costimulation modulators: a novel ap...
1,14967461,Emerging role of epidermal growth factor recep...
2,23468099,Effects of chronic social defeat stress on beh...
3,23293962,Hepatocyte growth factor activator inhibitor t...
4,7678677,Alprenolol and bromoacetylalprenololmenthane a...


In [8]:
def make_conll(text:pd.DataFrame, entity:pd.DataFrame,
               save_tag:bool=None,
               save_conll:bool=None,
               verbose:bool=None,
               begin_deviation:int=0,
               end_deviation:int=0 )->str:

    df_text = text.iloc[:,[0,1]]
    df_entity = entity.iloc[:,[0,1,2,3,4]]
    df_text.columns = ['text_id','text']
    df_entity.columns = ['text_id','begin','end','chunk','entity']
    entity_list = list(df_entity.entity.unique())


    ########--------------1.tag transformation function------------########

    def transform_text(text, entities, verbose=None):

        tag_list=[]
        for entity in entities.iterrows():

            begin = entity[1][1] + begin_deviation
            end = entity[1][2] + end_deviation
            chunk = entity[1][3]
            tag = entity[1][4]
            text = text[:end] + f' </END_NER:{tag}> ' + text[end:]
            text = text[:begin] + f' <START_NER:{tag}> ' + text[begin:]
            tag_list.append(tag)

        sum_of_added_entity = Counter(tag_list)
        sum_of_entity = Counter(entities['entity'].values)

        if verbose:
            print(f'Processed text id   : {entities.text_id.values[:1]}')
            print(f'Original Entities   : {sum_of_entity}\nAdded Entities      : {sum_of_added_entity}')
            print(f'Number Equality     : {sum_of_added_entity == sum_of_entity}')
            print("=="*40)

        if not sum_of_entity == sum_of_added_entity:
            print("There is a problem in text id:")
            print(entities.text_id.values[0])
            raise Exception("Check this text!")

        return text


    ######---------------2.apply_transform_text function ----------------#######

    def apply_tag_ner(df_text, df_entity, save=None, verbose=None):

        for text_id in tqdm(df_text.text_id):
            text  = df_text.loc[df_text['text_id']==text_id]['text'].values[0]
            entities  = df_entity.loc[(df_entity['text_id']==text_id)].sort_values(by='begin',ascending=False)

            df_text.loc[df_text['text_id']==text_id, 'text'] = transform_text(text, entities, verbose=verbose)

        if save:
            df_text.to_csv("text_with_ner_tag.csv", index=False, encoding='utf8')

        return df_text


    ##########----------------3.RUNNING TAG FUNCTION---------------#############

    print("Text tagging starting. Applying entities to whole text...\n")
    df = apply_tag_ner(df_text, df_entity, save=save_tag, verbose=verbose)


    ###########---------------4.Spark Pipeline-----------------------###########

    def spark_pipeline(df):
        spark_df = spark.createDataFrame(df)

        documentAssembler = nlp.DocumentAssembler()\
            .setInputCol("text")\
            .setOutputCol("document")\
            .setCleanupMode("shrink")

        sentenceDetector = nlp.SentenceDetector()\
            .setInputCols(['document'])\
            .setOutputCol('sentences')\
            .setExplodeSentences(True)

        tokenizer = nlp.Tokenizer() \
            .setInputCols(["sentences"]) \
            .setOutputCol("token")

        nlpPipeline = nlp.Pipeline(stages=[documentAssembler, sentenceDetector, tokenizer ])

        empty_df = spark.createDataFrame([['']]).toDF("text")
        pipelineModel = nlpPipeline.fit(empty_df)

        result = pipelineModel.transform(spark_df.select(['text']))


        return result.select('token.result').toPandas()
    print("\n\nSpark pipeline is running...")
    df_final = spark_pipeline(df)


    #########--------------5.CoNLL Function--------------------#############

    def build_conll(df_final, tag_list, save=None):

        header = "-DOCSTART- -X- -X- O\n\n"
        conll_text = ""
        chunks = []
        tag_list = tag_list
        tag = 'O'      # token tag
        ct = 'B'       # chunk tag part B or I

        for sentence_tokens in tqdm(df_final.result[:]):
            for token in sentence_tokens:
                if token.startswith("<START_NER:"):
                    tag = token.split(':')[1][:-1]
                    if tag not in tag_list:
                        tag = 'O'
                        conll_text += f'{token} NN NN {tag}\n'

                    continue

                if token.startswith("</END_NER:") and tag != 'O':
                    for i, chunk in enumerate(chunks):
                        ct = 'B' if i == 0 else 'I'
                        conll_text += f'{chunk} NNP NNP {ct}-{tag}\n'

                    chunks=[]
                    tag='O'
                    continue

                if tag != 'O':
                    chunks.append(token)
                    continue

                if tag == 'O':
                    conll_text += f'{token} NN NN {tag}\n'
                    continue

            conll_text += '\n'

        if save:
            with open("conll2003_text_file.conll", "w+", encoding='utf8') as f:
                f.write(header)
                f.write(conll_text)

        print("\nDONE!")
        return conll_text


    ########----------------6.RUNNING CONLL FUNCTION--------------------########

    print("Conll file is being created...\n")
    return build_conll(df_final, tag_list=entity_list, save=save_conll)


## Running the Create CoNLL Function

In [9]:
from tqdm import tqdm
from collections import Counter

In [10]:
# if you want tagged text or conll file saved in the current directory: just make default 'save_tag' or 'save_conll' parameters True.
conll_text = make_conll(train_text_df, train_entities_df, save_conll=True)

Text tagging starting. Applying entities to whole text...



100%|██████████| 1020/1020 [00:07<00:00, 132.52it/s]




Spark pipeline is running...
Conll file is being created...



100%|██████████| 10835/10835 [00:00<00:00, 27439.49it/s]



DONE!


In [11]:
# Checking conll string
print(conll_text[:532])

Selective NN NN O
costimulation NN NN O
modulators NN NN O
: NN NN O
a NN NN O
novel NN NN O
approach NN NN O
for NN NN O
the NN NN O
treatment NN NN O
of NN NN O
rheumatoid NN NN O
arthritis NN NN O
. NN NN O

T NN NN O
cells NN NN O
have NN NN O
a NN NN O
central NN NN O
role NN NN O
in NN NN O
the NN NN O
orchestration NN NN O
of NN NN O
the NN NN O
immune NN NN O
pathways NN NN O
that NN NN O
contribute NN NN O
to NN NN O
the NN NN O
inflammation NN NN O
and NN NN O
joint NN NN O
destruction NN NN O
characteristic NN NN O



### Saving CoNLL File

In [12]:
with open("conll2003_text_file.conll", "w+", encoding='utf8') as f:
    f.write("-DOCSTART- -X- -X- O\n\n")
    f.write(conll_text)

# Reading CoNLL File by Using CoNLL Reader

In [13]:
data = nlp.CoNLL().readDataset(spark, "./conll2003_text_file.conll")
data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Selective costimu...|[{document, 0, 96...|[{document, 0, 96...|[{token, 0, 8, Se...|[{pos, 0, 8, NN, ...|[{named_entity, 0...|
|T cells have a ce...|[{document, 0, 17...|[{document, 0, 17...|[{token, 0, 0, T,...|[{pos, 0, 0, NN, ...|[{named_entity, 0...|
|The requirement f...|[{document, 0, 22...|[{document, 0, 22...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|This approach is ...|[{document, 0, 78...|[{document, 0, 78...|[{token, 0, 3, Th...|[{pos, 0, 3, NN, ...|[{named_entity, 0...|
|it targets events...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 1, it...|[{pos, 0, 1, NN, ..